## Conversation Ending
### Goal

Can I use a model to tell me when a conversation has ended?

### Method

Start gathering up non-threaded messages in a channel. Use a UDF to call a LLM to determine if the conversation is continuing or new.

Use Kaskada to gather messages. Include up to 5 messages in the call to the LLM, independent of whether or not they are part of the conversation (as deemed by previous calls)


### The code:

#### Install the tools, initiate the things

In [ ]:
%pip install -q kaskada openai llama-cpp-python ipywidgets

In [ ]:
import openai, getpass

# Initialize OpenAI
openai.api_key = getpass.getpass('OpenAI: API Key')

In [ ]:
import pandas as pd
import kaskada as kd

# Initialize Kaskada with a local execution context.
kd.init_session()

# set pandas to display all floats with 6 decimal places
pd.options.display.float_format = '{:.6f}'.format

#### Pull in the user list, create a `format_user()` method

In [ ]:
users_df = pd.read_json("slack-generation.users.json")

columns_to_keep = ["id", "team_id", "name", "deleted", "real_name", "is_bot", "updated"]

users_df.drop(columns=users_df.columns.difference(columns_to_keep), inplace=True)

users = {}
for user in users_df.to_dict(orient='index').values():
    users[user["id"]] = user

In [ ]:
def get_user(user_id):
    return users[user_id] if user_id in users.keys() else None

def format_user(user_id):
    user = get_user(user_id)
    return f"{user['name']} ({user_id})" if user else f"({user_id})"

format_user("UBB9D2B01")

#### Load the slack data, clean the message text, format message users

In [ ]:
# Load events from a Parquet file
#
# if you wan to load in your own slack data, change this to the path of your output file from 1.1 above
# otherwise continue with `slack-generation.parquet`, which contains generated slack data for
# example purposes. See the `slack-generation/notebook.ipynb` notebook for more info.
input_file = "slack-generation.parquet"

# Use the "ts" column as the time associated with each row,
# and the "channel" column as the entity associated with each row.
raw_msgs = await kd.sources.Parquet.create(
    input_file,
    time_column = "ts",
    key_column = "channel",
    time_unit = "s"
)
raw_msgs.preview(5)

In [ ]:
@kd.udf("f<N: any>(x: N) -> string")
def format_users(batch: pd.Series):
    # Apply to each row in the batch
    return batch.map(format_user)

In [ ]:
# Clean Text
import re

def strip_code_blocks(line):
    return re.sub(r"```.*?```", '', line)

def user_repl(match_obj):
    user_id = match_obj.group(1)
    return format_user(user_id)

def update_users(line):
    return re.sub(r"<@(.*?)>", user_repl, line)

def clean_message(text):
        text = strip_code_blocks(update_users(text)).strip()
        return None if text == "" else text

@kd.udf("f<N: any>(x: N) -> string")
def clean_text(batch: pd.Series):
    # Apply to each row in the batch
    return batch.map(clean_message)

In [ ]:
formatted_msgs = raw_msgs.extend({
    "text": raw_msgs.col("text").pipe(clean_text),
    "user": raw_msgs.col("user").pipe(format_users)
})
formatted_msgs.preview(5)

#### Try to use a LLM to determine if a non-threaded message is part of the previous conversation

In [ ]:
thread_ts = formatted_msgs.col("thread_ts")

# split messages into two subgroups: threads and non-threads
non_threads = formatted_msgs.filter(thread_ts.is_null())

# hack in a new, empty string column: `is_new`
non_threads = non_threads.extend({"is_new": non_threads.col("text").substring(0,0)})

#filter to one channel for now
non_threads = non_threads.filter(non_threads.col("channel").eq("Project"))

@kd.udf("f<N: any>(x: N) -> string")
def format_message(batch: pd.Series):
    def formatter(raw):
        return f"{raw['user']} --> {raw['text']}" # --> {raw['reactions']}"
    return batch.map(formatter)

# prefix message with user
non_threads = non_threads.extend({"text": non_threads.select("user", "text").pipe(format_message)})

# collect previous messages into groups of potential conversations
prev_messages = non_threads.lag(1)
collect_window = kd.windows.Since(prev_messages.col("is_new").eq("yes"))

non_threads = non_threads.extend({"conversation" : prev_messages.col("text").collect(max=5, window=collect_window)})

non_threads.preview()

##### Setup instructions and few-shot learning examples for the LLM

In [ ]:
system = """
You are a helpful assistant. You will be passed an existing conversation and a next
line. Your job is to determine if the next line is part of the existing conversation
or the start of a new conversation. You should respond `yes` if you think the line
is part a new conversation or `no` otherwise. No explanation is needed.

Lines of the conversation, and the next line, will be passed in plain text, where the
user and their text is separated by an arrow like this: `-->`.

The user field contains an username and an user_id in parenthesis, like this:
`name (U1292934)` the username is lowercase and could match names in the conversation
text in a case-insensitive way.

Inside a conversation, `---` characters on their own line indicate that the next line will
contain the text from the next user in the conversation. Conversations may contain no
lines. When this is the case, the next line should always be a new conversation.

The conversation will be prefixed by `Conversation:` on it's on line and the next line
will be prefixed by `Next Line:` on its own line.
"""

user_empty_convo = """
Conversation:


Next Line:
userc (UFB3DA5BF) --> Risk mitigation is indeed essential, especially when we are relying
on real-time data for our inventory tracking and resource allocation. Let's prioritize
this aspect and design our system to be resilient.
"""

assistant_empty_convo = "yes"

user_existing_convo = """
Conversation:
userc (UFB3DA5BF) --> Risk mitigation is indeed essential, especially when we are relying
on real-time data for our inventory tracking and resource allocation. Let's prioritize
this aspect and design our system to be resilient.
---
userf (UEA27BBFF) --> Scaling our system effectively will be crucial for accommodating
future growth. We should also keep an eye on performance metrics and fine-tune
our resource allocation strategies as needed.


Next Line:
userb (UBB9D2B01) --> Sounds good, UserC. A short break will be refreshing. I'll be
back in 15 minutes with more ideas for the next steps.
"""

assistant_existing_convo = "no"

user_new_convo = """
Conversation:
userc (UFB3DA5BF) --> Risk mitigation is indeed essential, especially when we are relying
on real-time data for our inventory tracking and resource allocation. Let's prioritize
this aspect and design our system to be resilient.
---
userb (UBB9D2B01) --> Sounds good, UserC. A short break will be refreshing. I'll be
back in 15 minutes with more ideas for the next steps.


Next Line:
userb (UBB9D2B01) --> Good afternoon, everyone! The topic of multi-cloud strategies
is intriguing. I'm excited to explore how it can help us achieve high availability
for our inventory tracking tool.
"""

assistant_new_convo = 'yes'

ai_prompt_messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": user_empty_convo},
        {"role": "assistant", "content": assistant_empty_convo},
        {"role": "user", "content": user_existing_convo},
        {"role": "assistant", "content": assistant_existing_convo},
        {"role": "user", "content": user_new_convo},
        {"role": "assistant", "content": assistant_new_convo},
      ]

##### Declare a function that checks if a conversation is "new" by calling a LLM

In [ ]:
import time

def check_if_new_conversation(existing_conversation: [str], next_line: str) -> str:
    user_text = "Conversation:\n" + "\n---\n".join(existing_conversation) + "\n\n\nNext Line:\n" + next_line

    prompt = ai_prompt_messages.copy()
    prompt.append({"role": "user", "content": user_text})

    attempts = 0
    while True:
      try:
        attempts += 1
        completion = openai.ChatCompletion.create(
          # model choices: gpt-4, gpt-4-32k, gpt-3.5-turbo, gpt-3.5-turbo-16k
          model="gpt-3.5-turbo",
          messages=prompt,
          temperature=0
        )
        return completion.choices[0].message.content
      except:
        if attempts > 3:
           return "timeout"
        time.sleep(attempts * 5)

##### Call the function from a Kaskada UDF via the `pipe()` functionality, and output the results as jsonl

In [ ]:
@kd.udf("f<N: any>(x: N) -> string")
def determine_if_new(batch: pd.Series):
    def is_it_new(raw):
        return check_if_new_conversation(raw["conversation"], raw["text"])
    return batch.map(is_it_new)

non_threads = non_threads.extend({"is_new": non_threads.select("conversation", "text").pipe(determine_if_new)})

df = non_threads.to_pandas()
df

In [ ]:
df.to_json("ConversationEnding_v1_results.jsonl", orient="records", lines=True)

## Also test if we can use text in a string column to determine the breakpoint of a conversation.

In [ ]:
# can we use the text in a column to determine the breakpoint of a conversation? YES

test = await kd.sources.JsonlFile.create(
    "ConversationEnding_v1_input.jsonl",
    time_column = "ts",
    key_column = "channel",
    time_unit = "s"
)
test.preview(5)

In [ ]:
from datetime import timedelta

ts = test.col("ts")
new_convo = test.col("new_convo")

is_new = new_convo.eq("yes")

# Eventually this will just be: `thread_ts = ts.first(window=kd.windows.Since(is_new, start="inclusive"))`
#
# However, `Since()` is currently exclusive on the start of the window, inclusive on the end.
# But we need inclusive on the start and exclusive on the end.
#
# The hack below does what we need until `Since()` provides additional options for inclusivity
shifted_non_threads = test.shift_by(timedelta(microseconds=0.001))
shifted_ts = test.lag(1).col("ts").first(window=kd.windows.Since(is_new))
thread_ts = ts.if_(is_new).else_(shifted_ts)

# create threads_ts column for non-threaded messages
non_threads_threads = test.extend({"thread_ts": thread_ts}).filter(ts.is_not_null().and_(thread_ts.is_not_null()))
non_threads_threads.filter(non_threads_threads.col("channel").eq("General")).preview(20)

### Yes, we can use text in a string column to determine the breakpoint of a conversation.